In [8]:
## required packages : biopython, micca ##
## chmod +x is required for FASTQC, vsearch #
import os, re, glob, shutil
from Bio import SeqIO

In [9]:
def append(input_fn,output_handle,fmt='fastq',sep='.',sample_name=None):
    ## Taken from MICCA append function ##
    def append_sample_name(records,sample_name):
        for record in records:
            record.id = '{0};sample={1}'.format(record.id,sample_name)
            try:
                description = record.description.split(None,1)[1]
            except IndexError:
                description = ''
            record.description = description
            yield record
    if sample_name is None:
        sample_name = os.path.basename(input_fn).split(sep)[0]
        sample_name = sample_name.replace('_merged','')
    sample_name_nows = re.sub('\s+','_',sample_name)
    records_in = SeqIO.parse(input_fn,fmt)
    records_out = append_sample_name(records_in,sample_name_nows)
    SeqIO.write(records_out,output_handle,fmt)

In [10]:
is_pairend = 'no'
input_dir =  '/home/petadimensionlab/Desktop/PRJNA320132'
output_dir =  '/home/petadimensionlab/Desktop/PRJNA320132'

In [11]:
#### configurations ####
is_filecheck = 'yes'
is_fastqc = 'yes'
is_trim = 'yes'
is_append = 'yes'
is_qualfilter = 'yes'
is_unique = 'yes'
is_sort = 'yes'
## softwares ##
vsearch = '/home/petadimensionlab/miniconda3/pkgs/vsearch-2.7.0-1/bin/vsearch'
fastqc = '/home/petadimensionlab/miniconda3/pkgs/fastqc-0.11.7-pl5.22.0_1/bin/fastqc'
Trimmomatic = '/home/petadimensionlab/miniconda3/pkgs/trimmomatic-0.36-5/share/trimmomatic-0.36-5/trimmomatic.jar'
## input / output file ##

config_file_name = 'config.csv'
## reference file ##
chimera_ref = '/home/petadimensionlab/ws/ref/chimeras/rRNA16S.gold.NAST_ALIGNED.fasta'
gg97fasta = '/home/petadimensionlab/ws/ref/gg_13_5_otus/rep_set/97_otus.fasta'
gg97tax = '/home/petadimensionlab/ws/ref/gg_13_5_otus/taxonomy/97_otu_taxonomy.txt'
## parameters ##
maxdiffs = 5 #10
minmergelen = 1
maxmergelen = 600
minseqlength = 50
maxee_rate = 0.5/100 #%
minsize = 2
thr = 0.97
threadsnum = 8
## file names ##
appended_file_name = 'appended.fastq'
filtered_file_name = 'filtered.fasta'
unique_file_name = 'unique.fasta'
sorted_file_name = 'sorted.fasta'
nonchimera_file_name = 'nonchimera.fasta'
centroids_file_name = 'centroids.fasta'
denovootu_file_name = 'denovootu.txt'
hits_file_name = 'hits.txt'
ggotuname_file_name = 'ggotu_name.txt'
ggotuID_file_name = 'ggotu.txt'

In [12]:
## file integrity check ##
if is_filecheck=='yes':
    if is_pairend=='yes':
        fwds = set(glob.glob('^*_1.fastq.gz'))
        revs = set(glob.glob('^*_2.fastq.gz'))
        ffiles = rfiles = set()
        config_file = os.path.join(input_dir,config_file_name)
        fr = open(config_file,'r').readlines()
        for line in fr:
            line = line.replace('\n','')
            tmp = line.split(',')
            sID = tmp[0]
            ffiles.add(tmp[1])
            rfiles.add(tmp[2])
        fwddiff = list(fwds-ffiles)
        revdiff = list(revs-rfiles)
        if len(fwddiff)>0 or len(revdiff)>0:
            msg = 'Some FASTQ files are lacking. Quit computation.'
            print( msg )
            quit()
        else:
            msg = 'All FASTQ files exist. Continue computation.'
            print( msg )
    else:
        fwds = set(glob.glob('^*.fastq.gz'))
        ffiles = set()
        config_file = os.path.join(input_dir,config_file_name)
        fr = open(config_file,'r').readlines()
        for line in fr:
            line = line.replace('\n','')
            tmp = line.split(',')
            sID = tmp[0]
            ffiles.add(tmp[1])
        fwddiff = list(fwds-ffiles)
        if len(fwddiff)>0:
            msg = 'Some FASTQ files are lacking. Quit computation.'
            print( msg )
            quit()
        else:
            msg = 'All FASTQ files exist. Continue computation.'
            print( msg )

All FASTQ files exist. Continue computation.


In [13]:
## Quality Check by fastqc ##
if is_fastqc=='yes':
    fastqc_dir = os.path.join(input_dir,'fastqc')
    if not os.path.exists(fastqc_dir):
        os.mkdir(fastqc_dir)
    if is_pairend=='yes':
        config_file = os.path.join(input_dir,config_file_name)
        fr = open(config_file,'r').readlines()
        for line in fr:
            line = line.replace('\n','')
            tmp = line.split(',')
            sID = tmp[0]
            fwdfile = os.path.join(input_dir,tmp[1])
            revfile = os.path.join(input_dir,tmp[2])
            cmd = '%s %s' % (fastqc,fwdfile)
            os.system(cmd)
            cmd = '%s %s' % (fastqc,revfile)
            os.system(cmd)
    else:
        config_file = os.path.join(input_dir,config_file_name)
        fr = open(config_file,'r').readlines()
        for line in fr:
            line = line.replace('\n','')
            tmp = line.split(',')
            sID = tmp[0]
            fn = tmp[1]
            cmd = '%s %s' % (fastqc,fn)
            os.system(cmd)
    qcfiles = glob.glob('*_fastqc*')
    for qcfile in qcfiles:
        before = os.path.join(input_dir,qcfile)
        after = os.path.join(fastqc_dir,qcfile)
        shutil.move(before,after)

In [14]:
## merge pair-end files ##
if is_pairend=='yes':
    input_fns = []
    config_file = os.path.join(input_dir,config_file_name)
    fr = open(config_file,'r').readlines()
    for line in fr:
        line = line.replace('\n','')
        tmp = line.split(',')
        sID = tmp[0]
        fwdfile = os.path.join(input_dir,tmp[1])
        revfile = os.path.join(input_dir,tmp[2])
        paired_fwdfile = os.path.join(input_dir,'paired_'+tmp[1])
        paired_revfile = os.path.join(input_dir,'paired_'+tmp[2])
        unpaired_fwdfile = os.path.join(input_dir,'unpaired_'+tmp[1])
        unpaired_revfile = os.path.join(input_dir,'unpaired_'+tmp[2])
        tmpfile = os.path.join(output_dir,sID+'_merged.fasta')
        input_fns.append(tmpfile)
        ## trimming by Trimmomatic ##
        if is_trim=='yes':
            cmd = 'java -jar %s PE %s %s %s %s %s %s SLIDINGWINDOW:4:15 MINLEN:%d' % (Trimmomatic,fwdfile,revfile,paired_fwdfile,unpaired_fwdfile,paired_revfile,unpaired_revfile,minseqlength)
            os.system(cmd)
            cmd = '%s -fastq_mergepairs %s --reverse %s -fastq_maxdiffs %s -fastq_minmergelen %s -fastq_maxmergelen %s -fastqout %s' % (vsearch,paired_fwdfile,paired_revfile,str(maxdiffs),str(minmergelen),str(maxmergelen),tmpfile)
            os.system(cmd)
        else:
            cmd = '%s -fastq_mergepairs %s --reverse %s -fastq_maxdiffs %s -fastq_minmergelen %s -fastq_maxmergelen %s -fastqout %s' % (vsearch,fwdfile,revfile,str(maxdiffs),str(minmergelen),str(maxmergelen),tmpfile)
            os.system(cmd)
else:
    input_fns = []
    config_file = os.path.join(input_dir,config_file_name)
    fr = open(config_file,'r').readlines()
    for line in fr:
        line = line.replace('\n','')
        tmp = line.split(',')
        sID = tmp[0]
        fn = tmp[1]
        #input_file = os.path.join(output_dir,sID+'.fastq.gz')
        input_file = os.path.join(output_dir,fn)
        ## trimming by Trimmomatic ##
        if is_trim=='yes':
            tmpfile = os.path.join(output_dir,sID+'_trimmed.fastq')
            cmd = 'java -jar %s SE %s %s SLIDINGWINDOW:4:15 MINLEN:%d' % (Trimmomatic,input_file,tmpfile,minseqlength)
            os.system(cmd)
            input_fns.append(tmpfile)
        else:
            cmd = 'gunzip %s' % (input_file)
            os.system(cmd)
            input_fn = input_file.replace('.gz','')
            input_fns.append(input_fn)

In [15]:
## append file ##
appended_file = os.path.join(output_dir,appended_file_name)
if is_append=='yes':
    if is_pairend=='yes':
        ## append merged-pair fastq files ##
        with open(appended_file,'w') as output_handle:
            for input_fn in input_fns:
                append(input_fn,output_handle,'fastq','.')
        output_handle.close()
        ## remove tmporary files ##
        tmpfastqfiles = glob.glob('*_merged.fastq')
        for item in tmpfastqfiles:
            os.remove(item)
    else:
        ## append fastq files ##
        with open(appended_file,'w') as output_handle:
            for input_fn in input_fns:
                append(input_fn,output_handle,'fastq','.')
        output_handle.close()
else:
    msg = 'Skip appending.'
    print( msg )

In [16]:
## filtering ##
filtered_file = os.path.join(output_dir,filtered_file_name)
if is_qualfilter=='yes':
    cmd = '%s --fastq_filter %s --fastaout %s --fastq_maxee_rate %s' % (vsearch,appended_file,filtered_file,str(maxee_rate))
    os.system(cmd)
else:
    msg = 'Skip filtering.'
    print( msg )

In [17]:
## get unique sequences ##
unique_file = os.path.join(output_dir,unique_file_name)
if is_unique=='yes':
    cmd = '%s --derep_fulllength %s --output %s --sizeout --minseqlength %s' % (vsearch,filtered_file,unique_file,str(minseqlength))
    os.system(cmd)
else:
    msg = 'Skip unifying.'
    print( msg )

In [18]:
## sorted sequences ##
sorted_file = os.path.join(output_dir,sorted_file_name)
if is_sort=='yes':
    cmd = '%s --sortbysize %s --output %s --minsize %s' % (vsearch,unique_file,sorted_file,str(minsize))
    os.system(cmd)
else:
    msg = 'Skip sorting.'
    print( msg )